# Dataset 3 - California Housing 

## Import Libraries 

In [1]:
import warnings
import sys
sys.path.insert(0, '../src')
warnings.filterwarnings('ignore')

In [ ]:
import train_test
import ABC_train_test
import californiaDataset
import network
import statsModel
import performanceMetrics
import dataset
import sanityChecks
import torch 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy.stats import norm
from torch.utils.data import Dataset,DataLoader 
from torch import nn

## Parameters

ABC-Generator parameters are as mentioned below: 
1. mean : 1 ($\beta \sim N(\beta^{*},\sigma)$ where $\beta^{*}$ are coefficients of statistical model) or 1 ($\beta \sim N(0,\sigma)$
2. std : $\sigma = 1, 0.1 , 0.01$ (standard deviation)

In [ ]:
#ABC Generator Parameters
mean = 1 
variance = 0.01

## Dataset

In [ ]:
X,Y = californiaDataset.california_data()
n_features = 8
n_samples= 20640

## Stats Model

In [ ]:
[coeff,y_pred] = statsModel.statsModel(X,Y)

## Common Training Parameters (GAN & ABC_GAN) 

In [ ]:
n_epochs = 1000
threshold = 2.8051
batch_size = n_samples

## GAN Model

In [ ]:
real_dataset = dataset.CustomDataset(X,Y)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import pycuda.driver as cuda
cuda.init()
## Get Id of default device
torch.cuda.current_device()
# 0
cuda.Device(0).name()

**Training GAN for n_epochs number of epochs** 

In [ ]:
generator = network.Generator(n_features+2).to(device)
discriminator = network.Discriminator(n_features+2).to(device)

criterion = torch.nn.BCELoss()
gen_opt = torch.optim.Adam(generator.parameters(), lr=0.01, betas=(0.5, 0.999))
disc_opt = torch.optim.Adam(discriminator.parameters(), lr=0.01, betas=(0.5, 0.999))

In [ ]:
print(generator)
print(discriminator)

In [ ]:
discLossG1,genLossG1 = train_test.training_GAN(discriminator,generator,disc_opt,gen_opt,real_dataset,batch_size, n_epochs,criterion,device)

In [ ]:
GAN1_metrics = train_test.test_generator(generator,real_dataset,device)

In [ ]:
sanityChecks.discProbVsError(real_dataset,discriminator,device)

**Training GAN until loss is less than threshold**

In [ ]:
generator2 = network.Generator(n_features+2).to(device)
discriminator2 = network.Discriminator(n_features+2).to(device)
criterion = torch.nn.BCELoss()
gen_opt = torch.optim.Adam(generator2.parameters(), lr=0.01, betas=(0.5, 0.999))
disc_opt = torch.optim.Adam(discriminator2.parameters(), lr=0.01, betas=(0.5, 0.999))

In [ ]:
discLossG2,genLossG2 =train_test.training_GAN_3(discriminator2,generator2,disc_opt,gen_opt,real_dataset,batch_size,threshold,criterion,device)

In [ ]:
GAN2_metrics=train_test.test_generator_2(generator2,real_dataset,device)

In [ ]:
sanityChecks.discProbVsError(real_dataset,discriminator2,device)

# ABC GAN Model

### Training the network
**Training ABC-GAN for n_epochs number of epochs** 

In [ ]:
gen = network.Generator(n_features+2).to(device)
disc = network.Discriminator(n_features+2).to(device)

criterion = torch.nn.BCELoss()
gen_opt = torch.optim.Adam(gen.parameters(), lr=0.01, betas=(0.5, 0.999))
disc_opt = torch.optim.Adam(disc.parameters(), lr=0.01, betas=(0.5, 0.999))

In [ ]:
discLossA1,genLossA1 = ABC_train_test.training_GAN(disc, gen,disc_opt,gen_opt,real_dataset, batch_size, n_epochs,criterion,coeff,mean,variance,device)

In [ ]:
ABC_GAN1_metrics=ABC_train_test.test_generator(gen,real_dataset,coeff,mean,variance,device)

#### Sanity Checks 

In [ ]:
sanityChecks.discProbVsError(real_dataset,disc,device)

**Training GAN until mse of y_pred is > 0.1 or n_epochs < 30000**

In [ ]:
gen2 = network.Generator(n_features+2).to(device)
disc2 = network.Discriminator(n_features+2).to(device)

criterion = torch.nn.BCELoss()
gen_opt = torch.optim.Adam(gen2.parameters(), lr=0.01, betas=(0.5, 0.999))
disc_opt = torch.optim.Adam(disc2.parameters(), lr=0.01, betas=(0.5, 0.999))

In [ ]:
discLossA2,genLossA2 = ABC_train_test.training_GAN_3(disc2,gen2,disc_opt,gen_opt,real_dataset,batch_size,threshold,criterion,coeff,mean,variance,device)

In [ ]:
ABC_GAN2_metrics=ABC_train_test.test_generator_2(gen2,real_dataset,coeff,mean,variance,device)

In [ ]:
sanityChecks.discProbVsError(real_dataset,disc2,device)

# Model Analysis

In [ ]:
performanceMetrics.modelAnalysis(GAN1_metrics,ABC_GAN1_metrics,GAN2_metrics,ABC_GAN2_metrics)

In [ ]:
performanceMetrics.plotTrainingLoss(discLossG1,genLossG1,discLossA1,genLossA1,discLossG2,genLossG2,discLossA2,genLossA2)